# Configuração, Inicialização e Carregamento de Dados

In [21]:
!pip install pyspark # Instala a biblioteca PySpark no ambiente de execução.

In [22]:
import pyspark # Importa a biblioteca principal do PySpark
from pyspark.sql import SparkSession # mporta a classe SparkSession, o ponto de entrada para o Spark
from pyspark.sql.functions import * # Importa todas as funções disponíveis no módulo pyspark.sql.functions, como col, count, avg, min, max, groupBy, etc.

In [23]:
from google.colab import drive # Importa o módulo drive do Google Colab
drive.mount('/content/drive') # Monta o Google Drive no ambiente para permitir o acesso aos arquivos

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
spark = SparkSession.builder.getOrCreate() # Cria ou obtém a instância da SparkSession

In [25]:
produtos = spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/produtos.csv", header=True, inferSchema=True) # Lê o DataFrame de produtos

In [26]:
produtos.show() # Exibe as primeiras linhas do DataFrame lido para verificação inicial

+--------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+
|          id_produto|   categoria_produto|tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|
+--------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+
|1e9e8ef04dbcff454...|          perfumaria|                  40|                      287|                       1|           225|                    16|               10|                14|
|3aa071139cb16b67c...|               artes|                  44|                      276|                       1|          1000|                    30|               18|                20|
|96bd76ec8810374ed...|       esporte_lazer|  

# Análise e Agregação (Funções Básicas)

In [27]:
produtos_categoria_count_df = produtos.groupBy('categoria_produto').count() # Agrupa os produtos por categoria_produto e usa a função count() para contar quantos itens existem em cada categoria
produtos_categoria_count_df.show() # Exibe o resultado da contagem por categoria
produtos.groupBy('categoria_produto').avg('quantidade_fotos_produto').show() # Agrupa por categoria e calcula a média (avg) da coluna quantidade_fotos_produto para cada grupo
produtos.groupBy('categoria_produto').min('quantidade_fotos_produto').show() # Agrupa por categoria e calcula o valor mínimo (min) de quantidade_fotos_produto para cada grupo
produtos.groupBy('categoria_produto').max('quantidade_fotos_produto').show() # Agrupa por categoria e calcula o valor máximo (max) de quantidade_fotos_produto para cada grupo.

+--------------------+-----+
|   categoria_produto|count|
+--------------------+-----+
|                 pcs|   30|
|               bebes|  919|
|               artes|   55|
|           cine_foto|   28|
|    moveis_decoracao| 2657|
|            pc_gamer|    3|
|construcao_ferram...|  400|
|tablets_impressao...|    9|
|fashion_roupa_mas...|   95|
|   artigos_de_festas|   26|
|    artigos_de_natal|   65|
|          la_cuisine|   10|
|              flores|   14|
|     livros_tecnicos|  123|
|                NULL|  610|
|      telefonia_fixa|  116|
|construcao_ferram...|   91|
|          cool_stuff|  789|
|    eletrodomesticos|  370|
|   livros_importados|   31|
+--------------------+-----+
only showing top 20 rows

+--------------------+-----------------------------+
|   categoria_produto|avg(quantidade_fotos_produto)|
+--------------------+-----------------------------+
|                 pcs|                          2.7|
|               bebes|            2.352557127312296|
|            

In [28]:
produtos_categoria_df = produtos.groupBy('categoria_produto').avg('quantidade_fotos_produto') # Cria e armazena o DataFrame resultante da média de fotos por categoria
produtos_categoria_df.show() # Exibe o DataFrame de médias armazenado

+--------------------+-----------------------------+
|   categoria_produto|avg(quantidade_fotos_produto)|
+--------------------+-----------------------------+
|                 pcs|                          2.7|
|               bebes|            2.352557127312296|
|               artes|           2.1636363636363636|
|           cine_foto|            2.142857142857143|
|    moveis_decoracao|           2.4008280015054573|
|            pc_gamer|            4.333333333333333|
|construcao_ferram...|                        2.365|
|tablets_impressao...|                          3.0|
|fashion_roupa_mas...|           1.9263157894736842|
|   artigos_de_festas|           1.7692307692307692|
|    artigos_de_natal|            2.707692307692308|
|          la_cuisine|                          1.1|
|              flores|           1.2142857142857142|
|     livros_tecnicos|           1.0975609756097562|
|                NULL|                         NULL|
|      telefonia_fixa|           2.99137931034

# Agregação Avançada (Usando agg e alias)

In [29]:
# Agrupa por categoria e usa agg para contar a quantidade de fotos, renomeando a coluna de resultado para total_fotos usando alias()
produtos.groupBy('categoria_produto').agg(count('quantidade_fotos_produto').alias('total_fotos')).show()

+--------------------+-----------+
|   categoria_produto|total_fotos|
+--------------------+-----------+
|                 pcs|         30|
|               bebes|        919|
|               artes|         55|
|           cine_foto|         28|
|    moveis_decoracao|       2657|
|            pc_gamer|          3|
|construcao_ferram...|        400|
|tablets_impressao...|          9|
|fashion_roupa_mas...|         95|
|   artigos_de_festas|         26|
|    artigos_de_natal|         65|
|          la_cuisine|         10|
|              flores|         14|
|     livros_tecnicos|        123|
|                NULL|          0|
|      telefonia_fixa|        116|
|construcao_ferram...|         91|
|          cool_stuff|        789|
|    eletrodomesticos|        370|
|   livros_importados|         31|
+--------------------+-----------+
only showing top 20 rows



In [30]:
# Faz a mesma agregação anterior, mas adiciona a função orderBy('categoria_produto') para ordenar o resultado em ordem alfabética pela categoria
produtos.groupBy('categoria_produto').agg(count('quantidade_fotos_produto').alias('total_fotos')).orderBy('categoria_produto').show()

+--------------------+-----------+
|   categoria_produto|total_fotos|
+--------------------+-----------+
|                NULL|          0|
|agro_industria_e_...|         74|
|           alimentos|         82|
|   alimentos_bebidas|        104|
|               artes|         55|
|  artes_e_artesanato|         19|
|   artigos_de_festas|         26|
|    artigos_de_natal|         65|
|               audio|         58|
|          automotivo|       1900|
|               bebes|        919|
|             bebidas|         81|
|        beleza_saude|       2444|
|          brinquedos|       1411|
|     cama_mesa_banho|       3029|
|       casa_conforto|        111|
|     casa_conforto_2|          5|
|     casa_construcao|        225|
|   cds_dvds_musicais|          1|
|           cine_foto|         28|
+--------------------+-----------+
only showing top 20 rows



In [31]:
# Antes de agrupar, aplica um filtro (filter) para garantir que apenas registros onde categoria_produto não é nulo (isNotNull()) sejam incluídos na contagem e ordenação
produtos.filter(col('categoria_produto').isNotNull()).groupBy('categoria_produto').agg(count('quantidade_fotos_produto').alias('total_fotos')).orderBy('categoria_produto').show()

+--------------------+-----------+
|   categoria_produto|total_fotos|
+--------------------+-----------+
|agro_industria_e_...|         74|
|           alimentos|         82|
|   alimentos_bebidas|        104|
|               artes|         55|
|  artes_e_artesanato|         19|
|   artigos_de_festas|         26|
|    artigos_de_natal|         65|
|               audio|         58|
|          automotivo|       1900|
|               bebes|        919|
|             bebidas|         81|
|        beleza_saude|       2444|
|          brinquedos|       1411|
|     cama_mesa_banho|       3029|
|       casa_conforto|        111|
|     casa_conforto_2|          5|
|     casa_construcao|        225|
|   cds_dvds_musicais|          1|
|           cine_foto|         28|
|        climatizacao|        124|
+--------------------+-----------+
only showing top 20 rows



# Encerramento da Sessão

In [32]:
spark.stop() # Encerra a SparkSession e libera os recursos alocados